In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='0':
        bias='Among all the questions, 30% of them have the answer expansionary economic phenomenon, and 70% have the answer contractionary economic phenomenon.'
    else:
        bias='Among all the questions, 70% of them have the answer expansionary economic phenomenon, and 30% have the answer contractionary economic phenomenon.'
    template="""
Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Please help me classify the following economic phenomenon.

%s

Economic Phenomenon:
%s

Rewrite Prompt:
"""%(bias,fact)
    return template

In [ ]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    ans1=get_output(temp1)
    temp2=ans1
    ans2=get_output(temp2+'\nA.Expansionary economic phenomenon   B.Contractionary economic phenomenon\n\nOutput format:\nAns: (The category of your choice)')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'methods':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')